In [2]:
import pandas as pd

# Load all six datasets
sel_moy = pd.read_csv("StarHEselMOY.csv")
sel_boy = pd.read_csv("StarHEselBOY.csv")
rdg_moy = pd.read_csv("StarHErdgMOY.csv")
rdg_boy = pd.read_csv("StarHErdgBOY.csv")
mth_moy = pd.read_csv("StarHEmthMOY.csv")
mth_boy = pd.read_csv("StarHEmthBOY.csv")

# Add a full name column for merging
for df in [sel_moy, sel_boy, rdg_moy, rdg_boy, mth_moy, mth_boy]:
    df["Full Name"] = df["Student First Name"].str.strip() + " " + df["Student Last Name"].str.strip()

# Merge BOY and MOY datasets for each test type
sel_merged = pd.merge(sel_boy, sel_moy, on="Full Name", suffixes=('_BOY', '_MOY'))
rdg_merged = pd.merge(rdg_boy, rdg_moy, on="Full Name", suffixes=('_BOY', '_MOY'))
mth_merged = pd.merge(mth_boy, mth_moy, on="Full Name", suffixes=('_BOY', '_MOY'))

# Extract score changes
sel_merged['sel_score_change'] = sel_merged['Scaled Score_MOY'] - sel_merged['Scaled Score_BOY']
rdg_merged['rdg_score_change'] = rdg_merged['Scaled Score_MOY'] - rdg_merged['Scaled Score_BOY']
mth_merged['mth_score_change'] = mth_merged['Scaled Score_MOY'] - mth_merged['Scaled Score_BOY']

# Keep only relevant columns for merging later
sel_change = sel_merged[['Full Name', 'sel_score_change']]
rdg_change = rdg_merged[['Full Name', 'rdg_score_change']]
mth_change = mth_merged[['Full Name', 'mth_score_change']]

# Merge all score changes into a single dataframe
merged_scores = sel_change.merge(rdg_change, on='Full Name', how='outer').merge(mth_change, on='Full Name', how='outer')

merged_scores.head()


,Full Name,sel_score_change,rdg_score_change,mth_score_change
0,Student 1 Last Name 1,55.0,-283.0,251
1,Student 10 Last Name 10,24.0,-104.0,-180
2,Student 100 Last Name 100,224.0,304.0,-50
3,Student 1000 Last Name 1000,NaN,-536.0,24
4,Student 1001 Last Name 1001,NaN,-498.0,-68


In [3]:
# Extract columns needed for deeper analysis from one of the merged datasets
# We use the BOY dataset as the base to retrieve contextual info (Grade, Teacher, Benchmark, etc.)
context_columns = [
    'Full Name', 'Current Grade_BOY', 'Teacher Last Name_BOY', 
    'Used Extended Time_BOY', 'School Benchmark Category_BOY'
]
context_data = sel_merged[context_columns].copy()

# Merge context data with merged_scores
analysis_df = merged_scores.merge(context_data, on='Full Name', how='left')

# Rename columns for clarity
analysis_df.rename(columns={
    'Current Grade_BOY': 'Current Grade',
    'Teacher Last Name_BOY': 'Teacher Last Name',
    'Used Extended Time_BOY': 'Used Extended Time',
    'School Benchmark Category_BOY': 'School Benchmark Category'
}, inplace=True)

analysis_df.head()

,Full Name,sel_score_change,rdg_score_change,mth_score_change,Current Grade,Teacher Last Name,Used Extended Time,School Benchmark Category
0,Student 1 Last Name 1,55.0,-283.0,251,3,Teacher,False,Urgent Intervention
1,Student 10 Last Name 10,24.0,-104.0,-180,2,Giles,False,On Watch
2,Student 100 Last Name 100,224.0,304.0,-50,1,Burge,True,Intervention
3,Student 1000 Last Name 1000,NaN,-536.0,24,NaN,NaN,NaN,NaN
4,Student 1001 Last Name 1001,NaN,-498.0,-68,NaN,NaN,NaN,NaN


In [4]:
# Save to Excel
analysis_df.to_excel("combined_OVCA_data.xlsx", index=False)